# Proyecto (análisis dataset)

In [15]:
# --- General ---
import os

# --- Data ---
import pandas as pd
# from pandasql import sqldf
# import numpy as np
# import statistics as stats

# --- Conexión ---
# import elasticsearch

# --- Procesamiento lenguaje: spacy ---
import spacy
from spacy.matcher import Matcher
# from spacy.matcher import PhraseMatcher

# --- Procesamiento lenguaje: gensim ---
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel

# --- Prediccion Positividad: POS, NEU, NEG ---
from tqdm import tqdm
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# --- Visualización ---
import pyLDAvis
import pyLDAvis.gensim_models 
# import matplotlib.colors as colors
# import matplotlib.pyplot as plt
# import geopandas as gpd
%matplotlib inline

# ==============================================================================

# --- Configuración ---
# User password
password = os.environ.get('SOPHIA2')

# - - - geoData - - -
# Import tools_region.py
import tools_region as tr

# --- Funciones ---
# Cargar paquete de español mediano
nlp = spacy.load("es_core_news_md")

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Utilización de regiones y comunas

In [16]:
regiones = tr.get_regiones()
tr.list_regions(regiones)

Arica_y_Parinacota
Tarapacá
Antofagasta
Atacama
Coquimbo
Valparaíso
Región_del_Libertador_Gral._Bernardo_O’Higgins
Región_del_Maule
Región_de_Ñuble
Región_del_Biobío
Región_de_la_Araucanía
Región_de_Los_Ríos
Región_de_Los_Lagos
Región_Aisén_del_Gral._Carlos_Ibáñez_del_Campo
Región_de_Magallanes_y_de_la_Antártica_Chilena
Región_Metropolitana_de_Santiago


In [17]:
data_regiones_comunas = tr.search_region('Región_de_Los_Lagos')
if(data_regiones_comunas):
    region = data_regiones_comunas['region']
    comunas = data_regiones_comunas['comunas']

Region encontrada


## Leemos la información de los archivos

In [18]:
# --- Cargar Archivo ---
nombre_archivo = 'data_Region_X_EMPLEO_comunas_' # Resultado: ./data/{nombre_archivo}_{words}_{from_}_{to_}.csv

keyword="empleo empleos trabajo trabajos empleabilidad contratación"
simple_keyword= True

from_="2022-01-01"
to_="2022-06-30"

In [19]:
nombre_archivo = nombre_archivo+from_+"_"+to_
print(nombre_archivo)
df = pd.read_csv('./data/'+nombre_archivo+'.csv')
print(f'Largo noticias: {len(df)}')
df.drop('Unnamed: 0', axis=1, inplace=True)

df.head(2)

data_Region_X_EMPLEO_comunas_2022-01-01_2022-06-30
Largo noticias: 1642


,id_news,country,media_outlet,url,title,text,date,San_Pablo,San_Juan_de_la_Costa,Río_Negro,...,Frutillar,Fresia,Dalcahue,Curaco_de_Vélez,Cochamó,Chonchi,Chaitén,Castro,Calbuco,Ancud
0,47316734.0,chile,seminariolocal,https://www.semanariolocal.cl/archivos/6420,Parlamentario pide incentivar contratación de ...,Un aumento de mercados laborales poco dispuest...,2022-06-06,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,22598469.0,chile,elllanquihue,https://www.ellanquihue.cl/impresa/2022/05/03/...,En medio de inestabilidad económica preocupa p...,"Casi 415 mil personas estaban empleadas, en pr...",2022-05-03,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Definimos los patrones de búsqueda

In [20]:
# --- Texto de las noticias a lista ---
noticias = df.text.values.tolist()

nlp = spacy.load("es_core_news_sm")
matcher = Matcher(nlp.vocab)

# --- Definir patrones ---
# pattern: NOUN-de-NOUN
pattern_1 = [{"POS": "NOUN"},{"LOWER": "de"}, {"POS": "NOUN"}]
matcher.add("NOUN-de-NOUN", [pattern_1])

# pattern: NOUN-ADJ
pattern_2 = [{"POS": "NOUN"}, {"POS": "ADJ"}]
matcher.add("NOUN-ADJ", [pattern_2])

## Creamos filtro palabras relevantes

In [21]:
def text_to_list(noticia):
    list_of_words = []
    
    try:
        doc = nlp(noticia)

        for token in doc:
            if (token.pos_=="NOUN"):
                list_of_words.append(token.text)

        for ent in doc.ents:
            if (ent.label_ == "PER" and " " in ent.text):
                list_of_words.append(ent.text)

        matches = matcher(doc)

        for match_id, start, end in matches:
            span = doc[start:end]  # The matched span
            list_of_words.append(span.text)
    
    except Exception as e: 
        print(noticia)
        print(e)
    
    return list_of_words

## Procesamos las noticias

In [22]:
# --- Procesar texto ---
noticias_procesadas = []
for index, noticia in enumerate(noticias):
    noticia_procesada = text_to_list(noticia)
    noticias_procesadas.append(noticia_procesada)

## Prepararamos los datos de entrada de LDA
Los datos de entrada de LDA son: 
- un diccionario (variable 'id2word')
- nuestro dataset preprocesado ('noticias_procesadas')

In [23]:
# --- Crear diccionario de palabras ---
id2word = corpora.Dictionary(noticias_procesadas)
texts = noticias_procesadas

dataset_id = [id2word.doc2bow(noticia_procesada) for noticia_procesada in noticias_procesadas]

In [24]:
# --- Crear modelo LDA ---
lda_model = gensim.models.ldamodel.LdaModel(corpus=dataset_id,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [25]:

# --- Generar el mejor modelo LDA ---
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        print(num_topics)
        
        model = gensim.models.ldamodel.LdaModel(corpus=dataset_id,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
        
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [26]:
start=2
limit=10
step=2

In [27]:
# --- Optimal model ---
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=dataset_id, texts=noticias_procesadas, start=start, limit=limit, step=step)

2
4
6
8


In [28]:
# get max value
max_coh_value = -1
for index, coh_value  in enumerate(coherence_values):
    if coh_value > max_coh_value:
        max_coh_value = coh_value
        max_coh_index = index

In [29]:
print(f'Numero de topicos: {start + max_coh_index*step}')
optimal_model = model_list[max_coh_index]

Numero de topicos: 8


## Visualizar tópicos

In [30]:
# Visualize the topics
pyLDAvis.enable_notebook();
vis = pyLDAvis.gensim_models.prepare(optimal_model, dataset_id, id2word);
vis

/home/alex/miniconda3/envs/myenv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/alex/miniconda3/envs/myenv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/alex/miniconda3/envs/myenv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/alex/miniconda3/envs/myenv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/alex/miniconda3/envs/myenv/lib/python3.8/site-

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.373928  0.184037       1        1  79.785475
0      0.190638 -0.301961       2        1  12.470678
6     -0.085308  0.027477       3        1   2.054072
1     -0.098635  0.010579       4        1   1.974998
4     -0.102101  0.027279       5        1   1.868731
7     -0.109642  0.024893       6        1   1.235107
2     -0.088343  0.015260       7        1   0.343223
5     -0.080538  0.012435       8        1   0.267717, topic_info=                          Term         Freq        Total Category  logprob  \
399                    vecinos   321.000000   321.000000  Default  30.0000   
1877          establecimientos   241.000000   241.000000  Default  29.0000   
8                         años  1191.000000  1191.000000  Default  28.0000   
365                información   742.000000   742.000000  Default  27.0000   
770                      junio   222.000000   222.000000  Default  26.0000   
...                        ...          ...          ...      ...      ...   
1011         trimestre de 2019     0.074917     0.510955   Topic8  -8.9581   
971   comportamiento diferente     0.063117     0.499120   Topic8  -9.1295   
214                  trimestre     1.131217    20.110248   Topic8  -6.2435   
292          fuerza de trabajo     0.259386     3.487458   Topic8  -7.7162   
207                       tasa     0.508543    43.234878   Topic8  -7.0430   

      loglift  
399   30.0000  
1877  29.0000  
8     28.0000  
365   27.0000  
770   26.0000  
...       ...  
1011   4.0031  
971    3.8551  
214    3.0451  
292    3.3244  
207    1.4801  

[343 rows x 6 columns], token_table=       Topic      Freq             Term
term                                   
2876       5  0.929729             2023
21230      7  0.580314   Alfredo Torres
7014       5  0.986266    Antonio Varas
7419       7  0.776996  Eduardo Winkler
25366      2  0.996511        Joe Biden
...      ...       ...              ...
13361      2  0.992196            votos
2816       2  0.938995          víctima
2816       4  0.050303          víctima
34391      2  0.991303       zapatillas
84         1  0.999042             zona

[360 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 7, 2, 5, 8, 3, 6])

## Temas principales

In [31]:
def format_topics_documents(ldamodel=None, corpus=dataset_id, texts=texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['dominant_Topic', 'perc_Contribution', 'topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_documents(ldamodel=optimal_model, corpus=dataset_id, texts=noticias)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['document_No', 'dominant_Topic', 'topic_Perc_Contrib', 'keywords', 'text']
df_dominant_topic.head(5)

,document_No,dominant_Topic,topic_Perc_Contrib,keywords,text
0,0,3,0.9890,"trabajo, parte, personas, año, región, través,...",Un aumento de mercados laborales poco dispuest...
1,1,3,0.9555,"trabajo, parte, personas, año, región, través,...","Casi 415 mil personas estaban empleadas, en pr..."
2,2,3,0.9548,"trabajo, parte, personas, año, región, través,...",Luego que el Instituto Nacional de Estadística...
3,3,3,0.7300,"trabajo, parte, personas, año, región, través,...","Según la Encuesta Nacional de Empleo (ENE), qu..."
4,4,3,0.9905,"trabajo, parte, personas, año, región, través,...","En San Pablo, provincia de Osorno y en Chonchi..."


In [32]:
# add comuna to dataframe df_dominant_topic using merge
df_dominant_topic = pd.merge(df_dominant_topic, df)

## Revisar temas principales

In [33]:
df_dominant_topic.head(2)

,document_No,dominant_Topic,topic_Perc_Contrib,keywords,text,id_news,country,media_outlet,url,title,...,Frutillar,Fresia,Dalcahue,Curaco_de_Vélez,Cochamó,Chonchi,Chaitén,Castro,Calbuco,Ancud
0,0,3,0.9890,"trabajo, parte, personas, año, región, través,...",Un aumento de mercados laborales poco dispuest...,47316734.0,chile,seminariolocal,https://www.semanariolocal.cl/archivos/6420,Parlamentario pide incentivar contratación de ...,...,0,0,0,0,0,0,0,0,0,0
1,1,3,0.9555,"trabajo, parte, personas, año, región, través,...","Casi 415 mil personas estaban empleadas, en pr...",22598469.0,chile,elllanquihue,https://www.ellanquihue.cl/impresa/2022/05/03/...,En medio de inestabilidad económica preocupa p...,...,0,0,0,0,0,0,0,0,0,0


## Procesaremos la positividad de los datos (POS, NEU, NEG)

In [34]:
# --- Cargamos el modelo ---
model_name = "finiteautomata/beto-sentiment-analysis"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [35]:
# --- Funcion para obtener el sentimiento ---
df_dominant_topic['title_label'] = ""
df_dominant_topic['title_score'] = ""

for index, row in tqdm(df_dominant_topic.iterrows(), desc='Análisis de sentimiento', total=df_dominant_topic.shape[0]):
    
    # Analizamos su sentimiento en inglés
    sentiment_value = nlp(row['title'])
    
    # Insertamos en dataframe
    df_dominant_topic.at[index, "title_label"] = sentiment_value[0].get('label')
    df_dominant_topic.at[index, "title_score"] = sentiment_value[0].get('score')

Análisis de sentimiento: 100%|██████████| 1828/1828 [03:09<00:00,  9.64it/s]


In [36]:
df_dominant_topic.head(2)

,document_No,dominant_Topic,topic_Perc_Contrib,keywords,text,id_news,country,media_outlet,url,title,...,Dalcahue,Curaco_de_Vélez,Cochamó,Chonchi,Chaitén,Castro,Calbuco,Ancud,title_label,title_score
0,0,3,0.9890,"trabajo, parte, personas, año, región, través,...",Un aumento de mercados laborales poco dispuest...,47316734.0,chile,seminariolocal,https://www.semanariolocal.cl/archivos/6420,Parlamentario pide incentivar contratación de ...,...,0,0,0,0,0,0,0,0,NEU,0.989185
1,1,3,0.9555,"trabajo, parte, personas, año, región, través,...","Casi 415 mil personas estaban empleadas, en pr...",22598469.0,chile,elllanquihue,https://www.ellanquihue.cl/impresa/2022/05/03/...,En medio de inestabilidad económica preocupa p...,...,0,0,0,0,0,0,0,0,NEG,0.997927


## Guardamos nuevamente el dataset pero ahora procesado

In [37]:
df_dominant_topic.to_csv('./data/'+nombre_archivo+'_dominant_topic.csv', index=False)